# 06wk-1: 데이터분석 코딩패턴, 다중클래스 분류

최규빈  
2025-01-01

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/05wk-2
    .ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [6]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-yx6DQsLACqw8pWm0udv8Jm&si=in1eMD0-wU49y7mS >}}

# 2. Imports

In [20]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [21]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. 데이터분석 코딩패턴

## A. 일반적인 train/test 셋팅

`-` Step1: 데이터정리

In [103]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True)
to_tensor = torchvision.transforms.ToTensor()
X0 = torch.stack([to_tensor(img) for img, lbl in train_dataset if lbl==0])
X1 = torch.stack([to_tensor(img) for img, lbl in train_dataset if lbl==1])
X = torch.concat([X0,X1],axis=0).reshape(-1,784)
y = torch.tensor([0.0]*len(X0) + [1.0]*len(X1)).reshape(-1,1)
XX0 = torch.stack([to_tensor(img) for img, lbl in test_dataset if lbl==0])
XX1 = torch.stack([to_tensor(img) for img, lbl in test_dataset if lbl==1])
XX = torch.concat([XX0,XX1],axis=0).reshape(-1,784)
yy = torch.tensor([0.0]*len(XX0) + [1.0]*len(XX1)).reshape(-1,1)

`-` Step2: 학습가능한 오브젝트들의 설정 (모델링과정 포함)

`-` Step3: 학습 (=적합)

`-` Step4: 예측 & 결과분석

*train acc*

*test acc*

## B. Dropout 사용

`-` Step1: 데이터정리

In [92]:
pass

`-` Step2: 학습가능한 오브젝트들의 설정 (모델링과정 포함)

`-` Step3: 학습 (=적합)

`-` Step4: 예측 & 결과분석

*train acc*

*test acc*

## C. GPU도 사용

`-` Step1: 데이터정리

In [122]:
pass

`-` Step2: 학습가능한 오브젝트들의 설정 (모델링과정 포함)

`-` Step3: 학습 (=적합)

`-` Step4: 예측 & 결과분석

*train acc*

*test acc*

## D. 미니배치도 사용

`-` Step1: 데이터정리

`-` Step2: 학습가능한 오브젝트들의 설정 (모델링과정 포함)

`-` Step3: 학습 (=적합)

`-` Step4: 예측 & 결과분석

*train acc*

*test acc*

> 점점 **비본질적인** 코드가 늘어남 (=코드가 드럽다는 소리에요) –\>
> Trainer의 개념 등장

# 4. 다항분류

## A. 이항분류와 `BCEWithLogitsLoss`

## B. 범주형자료의 변환

`-` 범주형자료를 숫자로 어떻게 바꿀까?

-   실패 / 성공 $\to$ 0 / 1
-   숫자0그림 / 숫자1그림 $\to$ 0 / 1
-   남자 / 여자 $\to$ 0 / 1
-   강아지그림 / 고양이그림 $\to$ 0 / 1
-   강아지그림 / 고양이그림 / 토끼그림 $\to$ 0 / 1 / 2 ?????

`-` 주입식교육: 강아지그림/고양이그림/토끼그림일 경우 숫자화시키는 방법

-   잘못된방식: 강아지그림 = 0, 고양이그림 = 1, 토끼그림 = 2
-   올바른방식: 강아지그림 = \[1,0,0\], 고양이그림 = \[0,1,0\], 토끼그림
    = \[0,0,1\] \### \<– 이런방식을 원핫인코딩이라함

`-` 왜?

-   설명1: 강아지그림, 고양이그림, 토끼그림은 서열측도가 아니라
    명목척도임. 그래서 범주를 0,1,2 로 숫자화하면 평균과 같은 통계량들이
    의미가 없음 (사회조사분석사 2급 스타일의 설명)
-   설명2: 범주형은 원핫인코딩으로 해야함 (“30일만에 끝내는
    실전머신러닝” 이런 책에 나오는 스타일의 설명)
-   설명3: 동전을 한번 던져서 나오는 결과는 $n=1$인 이항분포를 따름.
    주사위 한번 던져서 나오는 눈금의 숫자는 $n=1$인 다항분포를 따름.
    $n=1$인 이항분포의 실현값은 0,1 이고, $n=1$인 다항분포의 실현값은
    \[1,0,0\], \[0,1,0\], \[0,0,1\] 이므로 당연히 $y_i$ 는 \[1,0,0\],
    \[0,1,0\], \[0,0,1\] 중 하나의 형태를 가진다고 가정하는게 바람직함
    (이 설명이 이 중에서 가장 정확한 설명임)

## C. 실습: 3개의 클래스를 구분¶

`-` 데이터준비

In [254]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
to_tensor = torchvision.transforms.ToTensor()
X0 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==0])
X1 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==1])
X2 = torch.stack([to_tensor(Xi) for Xi, yi in train_dataset if yi==2])
X = torch.concat([X0,X1,X2]).reshape(-1,1*28*28)
y = torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2)).reshape(-1,1).float()

## D. 결론 – 외우세여

`-` 파이토치버전 // 코딩용

|   분류   | netout의 의미 |      손실함수       |
|:--------:|:-------------:|:-------------------:|
| 이항분류 |     prob      |      `BCELoss`      |
| 이항분류 |     logit     | `BCEWithLogitsLoss` |
| 다항분류 |     probs     |         NA          |
| 다항분류 |    logits     | `CrossEntropyLoss`  |

> `CrossEntropyLoss` 이거 이름이 완전 마음에 안들어요..
> `CEWithLogitsLoss` 라고 하는게 더 좋을 것 같습니다.

`-` 일반적개념 // 이론용

|   분류   | 오차항의가정 | 마지막활성화함수 |       손실함수       |
|:--------:|:------------:|:----------------:|:--------------------:|
| 이항분류 |   이항분포   |    sigmoid[1]    | Binary Cross Entropy |
| 다항분류 |   다항분포   |    softmax[2]    |    Cross Entropy     |

[1] prob=sig(logit)

[2] probs=soft(logits)